In [23]:
## use the summary json file ot  answer questions based on the data 
## do embedding

In [24]:
from utils import get_openai_key


key = get_openai_key()

In [25]:
## open file 
from pathlib import Path
from dirs import *
import json

file_name = 'morepen_summary_32.json'
def load_summary(file_name)->dict:
    file_path = Path(SUMMARY_DIR,file_name)
    with open(file_path, 'r') as fr:
        return json.load(fr)
    
def load_summary_embedded(file_name)->dict:
    file_path = Path(EMBEDDING_DIR,file_name)
    with open(file_path, 'r') as fr:
        return json.load(fr)
    
def save_summary(doc, file_name):
     file_path =  Path(SUMMARY_DIR,file_name)
     with open(file_path, 'w') as fw:
        return json.dump(doc,fw)


In [26]:

doc = load_summary(file_name)
type(doc)

dict

In [27]:
from utils import get_openai_client
from utils_openai import get_embedding


In [28]:
# for k,v in doc.items():
#     doc[k]['embedding'] = get_embedding(client,doc[k]['chunk_text']) 

## save the embedded doc
# save_summary(doc,f'embedded_{file_name}')


In [29]:
## save the embedded doc
# save_summary(doc,f'embedded_{file_name}')

In [30]:
doc = load_summary_embedded(file_name)
client = get_openai_client()

In [31]:

question = "What was the overall performance of the company in the last quarter?"
question_embedding = get_embedding(client,question)


In [32]:

import numpy as np


def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Function to find top N most relevant chunks based on the question
def find_top_chunks(doc, question_embedding, top_n=3):
    similarities = []
    for key, value in doc.items():
        chunk_embedding = value['embedding']
        similarity = cosine_similarity(question_embedding, chunk_embedding)
        similarities.append((key, similarity))
    
    # Sort chunks by similarity in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Get the top N chunks
    top_chunks = [doc[key] for key, _ in similarities[:top_n]]
    return top_chunks


In [33]:
top_chunks = find_top_chunks(doc, question_embedding)

In [34]:
len(top_chunks)

3

In [35]:
summaries = [c['summary'] for c in top_chunks]
texts = [c['chunk_text'] for c in top_chunks]

In [36]:
texts

["r is usually slow in this business. Formulations have grown 2%. Seventy-one percent of the business comes from the medicine side, and 29% of the business comes from the OTC side. On the distribution side, we have more than 5,000 distributors, 5,100 distributors. We are present in 3 lakh outlets, 3.28 lakh outlets taken together. And all over the country, we are present. 580 people are there in all the states for the selling. The GenMix are distributed separately to a separate company. It was a single distributor. But otherwise, we have got service centers. We have got distributors all over the country. I'm not getting into detail of plans. Financial numbers have already been circulated. So as I shared that gross revenue during the quarter – I'm talking about the quarter now. During the quarter, the gross revenues have gone up 16.5%. EBITDA has gone up 182%, from 18 crores to 52 crores EBITDA. EBITDA level has gone from 5% to 12.31%. So it's more than double as a percentage because th

In [37]:
def create_prompt(top_chunks, question):
    prompt = "You are an expert analyst. Answer the following question based on the provided sections of a document.\n\n"
    
    for chunk in top_chunks:
        section = chunk['section']
        # summary = chunk['summary']
        texts = chunk['chunk_text']
        prompt += f"{section}: {texts}\n\n"
    
    prompt += f"Question: {question}\n"
    return prompt

In [38]:
prompt = create_prompt(top_chunks,question)

In [39]:
from constants import DEFAULT_TEMPERATURE, DEFAULT_TOP_P


system_content =  "You are a helpful assistant."
client = get_openai_client() 
def get_answer_from_gpt(client,
                        prompt,
                    system_content = system_content,
                    model = "gpt-3.5-turbo",
                    max_tokens = 150):
    


    response = client.chat.completions.create(
                                                model=model,
                                                messages=[
                                                    {"role": "system", "content":system_content},
                                                    {"role": "user","content":prompt}],
                                                temperature=DEFAULT_TEMPERATURE,
                                                max_tokens=max_tokens,
                                                top_p=DEFAULT_TOP_P
                                            )
    answer = response.choices[0].message.content
    return  answer

In [40]:
get_answer_from_gpt(client=client,prompt=prompt,)

'Based on the information provided, the overall performance of the company in the last quarter was strong. The gross revenues increased by 16.5%, EBITDA increased by 182%, EBITDA level increased from 5% to 12.31%, and profit before tax almost tripled from 3% to 9.88%. Profit after tax also showed significant growth, increasing by almost 259% from 2.25% to 6.73%. Additionally, earnings per share increased by 144%, return on equity doubled from 5% to 11%, and return on capital employed almost doubled from 7.31% to 15%. The company also achieved a 279% increase in value and saw improvements in various'

In [41]:
from constants import DEFAULT_MODEL


def classify_question(client,question):
    classification_prompt = f"I will provide you with a prompt given by a user to an AI agent trained on a document. Use your intelligence to understand the intent of the user and classify the question as either : SPECIFIC or SUMMARY. return either of the two words in response. Heres the question as given by user: \n\nQuestion: {question}\n"

    
    response = client.chat.completions.create(
                                                model=DEFAULT_MODEL,
                                                messages=[
                                                    {"role": "system", "content":system_content},
                                                    {"role": "user","content":classification_prompt}],
                                                temperature=DEFAULT_TEMPERATURE,
                                                max_tokens=300,
                                                top_p=DEFAULT_TOP_P
                                            )
    classification = response.choices[0].message.content.strip().lower()
    return classification


In [42]:
def create_summary_prompt(doc):
    print("creating summary prompt")
    prompt = "You are an expert analyst. Summarize the entire document based on the provided sections.\n\n"
    count = 0
    for key, chunk in doc.items():
        if count ==20:
            break
        else:
            section = chunk['section']
            summary = chunk['summary']
            prompt += f"{section}: {summary}\n\n"
            count +=1
    return prompt

In [43]:
# count_tokens(create_summary_prompt(doc))

In [44]:
from utils_openai import count_tokens


def answer_question(doc, question):
    client = get_openai_client()
    classification = classify_question(client, question)
    print(classification)
    if classification == "summary":
        
        prompt = create_summary_prompt(doc)
        print(count_tokens(prompt))
    else:
        question_embedding = get_embedding(client,question)
        top_chunks = find_top_chunks(doc, question_embedding,top_n=3)
        summaries = [c['summary'] for c in top_chunks]
        texts = [c['chunk_text'] for c in top_chunks]
        print(summaries)
        print(texts)
        prompt = create_prompt(top_chunks, question)
        print(count_tokens(prompt))
    answer = get_answer_from_gpt(client,prompt)
    return answer

In [45]:
question = "what are the key promises given by the management for the coming quarters? what metrics should be increasing to show growth?"
# question = "summarise the whole concall in details including various sections of the call and outlining the important financials etc. around 1000 words "

In [46]:
# question = "what is document about ?"
question = "what is this API you mentioned?"
question = "based on the quarterly data what do you think should I invest in this company for mid to near term"

In [ ]:
messages = [ { "role":"system" ,"content":"you are something"},
 { "role":"user" ,"content":"I will give you with some questions that a user will ask on the document of conference calls. Answer them accordingly."},
 
 ]
messages.append({ "role":"user" ,"content":question} )
answer = answer_question(doc, question)
messages.append({"role":"system","content":answer})
            

In [47]:

answer = answer_question(doc, question)
answer

specific
["In this chunk of the earnings conference call, the management provides an overview of the company's performance and operations. They mention that formulations have grown by 2% and discuss the distribution side, highlighting that they have over 5,100 distributors and are present in 3.28 lakh outlets across the country. The company has 580 sales representatives in all states. The management shares the financial performance of the quarter, stating that gross revenues increased by 16.5%, EBITDA increased by 182%, and profit before tax almost tripled. Profit after tax also saw a significant increase of 259%. On an annual basis, the company achieved a 20% revenue growth, 101% EBITDA growth, and significant improvements in various financial ratios such as return on equity and return on capital employed. The earnings per share increased by 144% to 1 rupee and 88 paisa, and the return on equity doubled to 11%. The company also maintained a high asset turnover ratio and a negligible e

'Based on the quarterly data provided in the document, it seems like the company has shown significant growth and improvement in various financial metrics such as revenue, EBITDA, profit before tax, profit after tax, return on equity, return on capital employed, and asset turnover ratio.\n\nThe company has reported a 16.5% increase in gross revenues during the quarter, with EBITDA increasing by 182% and profit before tax almost tripling. Additionally, the profit after tax has increased by 259% on a quarterly basis.\n\nOn an annual basis, the company has achieved a 20% revenue growth, 101% EBITDA growth, and significant improvements in other financial ratios such as return on equity and return on capital employed.\n\nConsidering'